In [2]:
!bq --location=US mk --dataset "datamart"

Dataset 'ilitzkyzhou:datamart' successfully created.


In [4]:
%%bigquery
  create or replace table datamart.title_akas as
    select titleId, ordering, title, region, language, types, attributes, isOriginalTitle from imdb_staging.title_akas

""


In [5]:
%%bigquery
  create or replace table datamart.name_basics as
    select * from imdb_staging.name_basics

""


In [36]:
%%bigquery
  update datamart.name_basics
    set birthYear='0' where birthYear='\\N'

""


In [37]:
%%bigquery
  update datamart.name_basics
    set deathYear='0' where deathYear='\\N'

""


In [39]:
%%bigquery
  create or replace table datamart.name_basics as 
    select nconst, primaryName, cast(birthYear as int64) as birthYear, cast(deathYear as int64) as deathYear, primaryProfession, knownforTitles
    from datamart.name_basics

""


In [25]:
# combine title basics, title crew, title ratings into a single table as they are all referencing the same entity

%%bigquery
  create or replace table datamart.title as
    select basics.tconst, basics.titleType, basics.primaryTitle, basics.originalTitle, basics.isAdult, basics.startYear, basics.endYear, basics.runTimeMinutes, basics.genres, crew.directors, crew.writers, ratings.averageRating, ratings.numVotes
    from imdb_staging.title_basics as basics join imdb_staging.title_crew as crew on basics.tconst=crew.tconst
    left outer join imdb_staging.title_ratings as ratings on basics.tconst=ratings.tconst

""


In [26]:
%%bigquery
  update datamart.title
    set startYear='0' where startYear='\\N'

""


In [31]:
%%bigquery
  update datamart.title
    set endYear='0' where endYear='\\N'

""


In [32]:
%%bigquery
  update datamart.title
    set runTimeMinutes='0' where runTimeMinutes='\\N'

""


In [34]:
%%bigquery
  create or replace table datamart.title as
    select tconst, titleType, primaryTitle, originalTitle, isAdult, cast(startYear as int64) as startYear, cast(endYear as int64) as endYear, cast(runTimeMinutes as int64) as runTimeMinutes, genres, directors, writers, averageRating, numVotes
    from datamart.title

""


In [17]:
%%bigquery
  create or replace table datamart.episode as 
    select * from imdb_staging.title_episode

""


In [25]:
%%bigquery
  create or replace table datamart.principals as 
    select * from imdb_staging.title_principals

""


In [123]:
%%bigquery
  create or replace table datamart.bollywood as
    select * from kaggle_staging.bollywood

""


In [22]:
%%bigquery
  create or replace table datamart.bollywood_actors as
    select * from kaggle_staging.bollywood_actors

""


In [23]:
%%bigquery
  create or replace table datamart.bollywood_actress as
    select * from kaggle_staging.bollywood_actress

""


In [41]:
%%bigquery
  select tconst from datamart.title group by tconst having count(*) > 1

,tconst


no duplicates

In [44]:
%%bigquery
  select titleId, ordering from datamart.title_akas group by titleId, ordering having count(*) > 1

,titleId,ordering


In [75]:
%%bigquery
  select nconst, tconst from imdb_staging.title_principals group by nconst, tconst having count(*) > 1

,nconst,tconst
0,nm4308499,tt12641434
1,nm0168035,tt0058792
2,nm1930143,tt1366977
3,nm2968661,tt0172614
4,nm11783375,tt13667528
...,...,...
1027,nm7861505,tt7776966
1028,nm1773375,tt2672634
1029,nm1141719,tt5098626
1030,nm4327231,tt12381026


In [72]:
%%bigquery
create or replace table datamart.principals as
  select tconst, ordering, nconst, category, job from (
  select *, row_number() over (partition by nconst, tconst) row_number from datamart.principals)
    where row_number=1

""


In [76]:
#duplicates removed
%%bigquery
  select nconst, tconst from datamart.principals group by nconst, tconst having count(*) > 1

,nconst,tconst


In [78]:
%%bigquery
  select tconst, parentTconst from datamart.episode group by tconst, parentTconst having count(*) > 1

,tconst,parentTconst


In [79]:
%%bigquery
  select nconst from datamart.name_basics group by nconst having count(*) > 1

,nconst


In [116]:
%%bigquery
select title, Director from datamart.bollywood group by title, director having count(*) > 1

,title,Director
0,Chandrasena,V. Shantaram
1,Sati Naag Kanya,Babubhai Mistri
2,Mera Yaar Mera Dushman,Unknown
3,Raaz,Unknown
4,Aparichit,S. Shankar
5,Mr. Bhatti on Chutti,Karan Razdan
6,Monsoon Shootout,Amit Kumar
7,Wah Taj,Ajit Sinha
8,Sultana Daku,Mohammed Hussain
9,Yakeen,Unknown


In [124]:
%%bigquery
create or replace table datamart.bollywood as
  select * from (
  select *, row_number() over (partition by Title, Director) row_number from datamart.bollywood)
    where row_number=1

""


In [125]:
%%bigquery
alter table datamart.bollywood drop column row_number

""


In [126]:
%%bigquery
select title, Director from datamart.bollywood group by title, director having count(*) > 1

,title,Director


In [83]:
%%bigquery
select name from datamart.bollywood_actors group by name having count(*) > 1

,name


In [84]:
%%bigquery
select name from datamart.bollywood_actress group by name having count(*) > 1

,name


In [30]:
%%bigquery
select p.nconst from datamart.principals p left outer join datamart.name_basics n on n.nconst=p.nconst where n.nconst is null

,nconst
0,nm8117851
1,nm1659538
2,nm4979449
3,nm9615033
4,nm8117851
...,...
27514,nm10449319
27515,nm10168357
27516,nm1110388
27517,nm8019227


In [35]:
%%bigquery
create or replace table datamart.principals as
select p.tconst, p.ordering, p.nconst, p.category, p.job from datamart.principals p left outer join datamart.name_basics n on n.nconst=p.nconst where n.nconst is not null

""


In [36]:
%%bigquery
select p.nconst from datamart.principals p left outer join datamart.name_basics n on n.nconst=p.nconst where n.nconst is null

#check that referential integrity violations have been removed

,nconst


In [37]:
%%bigquery
select p.tconst from datamart.principals p left outer join datamart.title t on t.tconst=p.tconst where t.tconst is null

,tconst
0,tt14447124
1,tt14447040
2,tt14450326
3,tt14446674
4,tt14449240
...,...
14733,tt14447910
14734,tt6030010
14735,tt14447910
14736,tt6798666


In [39]:
%%bigquery
create or replace table datamart.principals as
select p.tconst, p.ordering, p.nconst, p.category, p.job from datamart.principals p left outer join datamart.title t on t.tconst=p.tconst where t.tconst is not null

""


In [42]:
%%bigquery
select p.tconst from datamart.principals p left outer join datamart.title t on t.tconst=p.tconst where t.tconst is null

#check that referential integrity violations have been removed

,tconst


In [32]:
%%bigquery
select e.tconst from datamart.episode e left outer join datamart.title t on t.tconst=e.tconst where t.tconst is null

,tconst
0,tt14448056
1,tt14446626
2,tt14446958
3,tt14446168
4,tt14448078
...,...
2360,tt14445812
2361,tt14449324
2362,tt14445074
2363,tt14450746


In [43]:
%%bigquery
create or replace table datamart.episode as
select e.tconst, e.parentTconst, e.seasonNumber, e.episodeNumber from datamart.episode e left outer join datamart.title t on t.tconst=e.tconst where t.tconst is not null

""


In [44]:
%%bigquery
select e.tconst from datamart.episode e left outer join datamart.title t on t.tconst=e.tconst where t.tconst is null

#check that referential integrity violations have been removed

,tconst


In [45]:
%%bigquery
select e.tconst from datamart.episode e left outer join datamart.title t on e.parentTconst=t.tconst where t.tconst is null

,tconst
0,tt1909106
1,tt6452488
2,tt6452490
3,tt6411832
4,tt6420584
5,tt6452492
6,tt6452494
7,tt6411794
8,tt7321004
9,tt7355152


In [46]:
%%bigquery
create or replace table datamart.episode as
select e.tconst, e.parentTconst, e.seasonNumber, e.episodeNumber from datamart.episode e left outer join datamart.title t on t.tconst=e.parentTconst where t.tconst is not null

""


In [47]:
%%bigquery
select e.tconst from datamart.episode e left outer join datamart.title t on e.parentTconst=t.tconst where t.tconst is null

#check that referential integrity violations have been removed

,tconst


In [34]:
%%bigquery
select a.titleId from datamart.title_akas a left outer join datamart.title t on a.titleId=t.tconst where t.tconst is null

,titleId
0,tt4698454
1,tt2077873
2,tt1757913
3,tt14449018
4,tt14449018
...,...
15843,tt14438964
15844,tt6804308
15845,tt2193355
15846,tt14450682


In [48]:
%%bigquery
create or replace table datamart.title_akas as
select a.titleId, a.ordering, a.title, a.region, a.language, a.types, a.attributes, a.isOriginalTitle from datamart.title_akas a left outer join datamart.title t on a.titleId=t.tconst where t.tconst is not null

""


In [49]:
%%bigquery
select a.titleId from datamart.title_akas a left outer join datamart.title t on a.titleId=t.tconst where t.tconst is null

#check that referential integrity violations have been removed

,titleId


In [28]:
%%bigquery
select actress.name from datamart.bollywood_actress actress left outer join datamart.bollywood b on actress.Debut_aslead_role=b.title where b.title is null

,name
0,Farida Jalal
1,Nandita Das
2,Raakhee
3,Pooja Bhatt
4,Ayesha Jhulka
5,Asha Parekh
6,Warina Hussain
7,Reena Roy
8,Rati Agnihotri
9,Aishwarya Rai


In [52]:
%%bigquery
create or replace table datamart.bollywood_actress as
select actress.name, actress.Height_in_cm_, actress.Debut_aslead_role from datamart.bollywood_actress actress join datamart.bollywood b on actress.Debut_aslead_role=b.title

# I just realized that I could've just done an inner join this entire time :|

""


In [53]:
%%bigquery
select actress.name from datamart.bollywood_actress actress left outer join datamart.bollywood b on actress.Debut_aslead_role=b.title where b.title is null

#check that referential integrity violations have been removed

,name


Getting tv shows that contain the highest number of episodes assuming more than 10 episodes

In [1]:
%%bigquery
select parentTconst, count(tconst) as num_episodes from datamart.episode group by parentTconst having num_episodes > 10 order by num_episodes desc limit 10

,parentTconst,num_episodes
0,tt0058796,14269
1,tt0069658,12277
2,tt0053494,10205
3,tt0055708,9957
4,tt0270116,9858
5,tt1985601,9502
6,tt0068120,9071
7,tt0088580,8942
8,tt0283794,8935
9,tt0283767,8386


Getting tv shows whose average rating exceeds the average rating of its respective titleType

In [9]:
%%bigquery
SELECT tconst, originalTitle, titleType, averageRating from datamart.title t1 where averageRating > (select avg(averageRating) from datamart.title t2 where t1.titleType=t2.titleType) order by titleType

,tconst,originalTitle,titleType,averageRating
0,tt0156754,Maalik,movie,6.6
1,tt0046059,Meet Me at the Fair,movie,6.4
2,tt0032098,La vida de Carlos Gardel,movie,6.3
3,tt0021922,Good Sport,movie,6.7
4,tt0156873,Parthal Pasi Theerum,movie,7.4
...,...,...,...,...
630950,tt0433090,Teenage Mutant Ninja Turtles,videoGame,7.4
630951,tt2974350,Dairantou sumasshu burazâzu for Wii U,videoGame,8.5
630952,tt6143942,WRC 4: The Official Game of the FIA World Rall...,videoGame,7.5
630953,tt1485814,Up,videoGame,7.8


Getting average ratings of all titleTypes for titles that have been released post-1990

In [4]:
%%bigquery
select titleType, avg(averageRating) as averageRating from (select titleType, averageRating from datamart.title where startYear > 1990) group by titleType

,titleType,averageRating
0,tvShort,6.849872
1,tvSpecial,6.697889
2,videoGame,7.084054
3,tvMiniSeries,7.042440
4,episode,NaN
5,movie,6.178751
6,short,7.046083
7,video,6.619048
8,tvMovie,6.507392
9,tvSeries,6.731255


Getting average ratings and total votes of shows that are titles that are for adults vs titles that are not for adults, for all shows released post-2000.

In [6]:
%%bigquery
select isAdult, avg(averageRating) as averageRating, sum(numVotes) as totalAttention from (select isAdult, averageRating, numVotes from datamart.title where startYear > 2000) group by isAdult

,isAdult,averageRating,totalAttention
0,0,7.010993,754411356
1,1,6.631483,137631


Getting average rating of all titles per region

In [11]:
%%bigquery
select akas.region, avg(averageRating) as averageRating from datamart.title_akas akas join datamart.title title on title.tconst=akas.titleId group by akas.region having region != '\\N'

,region,averageRating
0,FR,6.685948
1,JP,6.896177
2,ES,6.726118
3,PT,6.913388
4,IN,6.921303
...,...,...
241,PW,4.800000
242,KI,NaN
243,DJ,6.300000
244,NU,5.800000


In [12]:
!bq --location=US mk --dataset "reports"

Dataset 'ilitzkyzhou:reports' successfully created.


In [14]:
%%bigquery
create view reports.rating_by_type as
select titleType, avg(averageRating) as averageRating from (select titleType, averageRating from datamart.title where startYear > 1990) group by titleType

""


In [15]:
%%bigquery
create view reports.rating_by_region as
select akas.region, avg(averageRating) as averageRating from datamart.title_akas akas join datamart.title title on title.tconst=akas.titleId group by akas.region having region != '\\N'

""
